In [ ]:
import numpy as np
from numba import jit
from rgi.rgizero.games.connect4 import Connect4Game, Connect4State, PlayerId, Action
from rgi.rgizero.games import connect4

# Create a Connect4Game instance
game = Connect4Game()

# Create a sample game state (e.g., a partially filled board)
board = np.zeros((game.height, game.width), dtype=np.int8)
board[5, 0] = 1 # Player 1 makes a move
board[4, 0] = 2 # Player 2 makes a move
board[3, 0] = 1 # Player 1 makes a move
board[5, 1] = 2
board[4, 1] = 1
board[3, 1] = 2
board[5, 2] = 1
board[4, 2] = 2
current_player = 1
initial_state = Connect4State(board=board, current_player=current_player, is_terminal=False, winner=None)

# Non-jitted versions for comparison
def py_calculate_winner(board: np.ndarray, col: int, row: int, player: PlayerId, height: int, width: int, connect_length: int) -> PlayerId | None:
    directions = [
        (0, 1),  # Horizontal
        (1, 0),  # Vertical
        (1, 1),  # Diagonal /
        (1, -1),  # Diagonal \
    ]

    for dr, dc in directions:
        count = 1
        for factor in [-1, 1]:
            r, c = row + dr * factor, col + dc * factor
            while 0 <= r < height and 0 <= c < width and board[r, c] == player:
                count += 1
                r, c = r + dr * factor, c + dc * factor
                if count >= connect_length:
                    return player

    return None

def py_calculate_is_board_full(board: np.ndarray) -> bool:
    return np.all(board != 0).item()


print("Warming up")
# Warm-up Numba & python functions
connect4._numba_calculate_winner(initial_state.board, 0, 2, current_player, game.height, game.width, game.connect_length)
connect4._numba_calculate_is_board_full(initial_state.board)
py_calculate_winner(initial_state.board, 0, 2, current_player, game.height, game.width, game.connect_length)
py_calculate_is_board_full(initial_state.board)

print("Profiling _calculate_winner:")
%timeit connect4._numba_calculate_winner(initial_state.board, 0, 2, current_player, game.height, game.width, game.connect_length)
%timeit py_calculate_winner(initial_state.board, 0, 2, current_player, game.height, game.width, game.connect_length)

print("\nProfiling _calculate_is_board_full:")
%timeit connect4._numba_calculate_is_board_full(initial_state.board)
%timeit py_calculate_is_board_full(initial_state.board)


import numpy as np
from numba import jit
from rgi.rgizero.games.connect4 import Connect4Game, Connect4State, PlayerId, Action

# Create a Connect4Game instance
game = Connect4Game()

# Create a sample game state (e.g., a partially filled board)
board = np.zeros((game.height, game.width), dtype=np.int8)
board[5, 0] = 1 # Player 1 makes a move
board[4, 0] = 2 # Player 2 makes a move
board[3, 0] = 1 # Player 1 makes a move
board[5, 1] = 2
board[4, 1] = 1
board[3, 1] = 2
board[5, 2] = 1
board[4, 2] = 2
current_player = 1
initial_state = Connect4State(board=board, current_player=current_player, is_terminal=False, winner=None)

# Non-jitted versions for comparison
def py_calculate_winner(board: np.ndarray, col: int, row: int, player: PlayerId, height: int, width: int, connect_length: int) -> PlayerId | None:
    directions = [
        (0, 1),  # Horizontal
        (1, 0),  # Vertical
        (1, 1),  # Diagonal /
        (1, -1),  # Diagonal \
    ]

    for dr, dc in directions:
        count = 1
        for factor in [-1, 1]:
            r, c = row + dr * factor, col + dc * factor
            while 0 <= r < height and 0 <= c < width and board[r, c] == player:
                count += 1
                r, c = r + dr * factor, c + dc * factor
                if count >= connect_length:
                    return player

    return None

def py_calculate_is_board_full(board: np.ndarray) -> bool:
    return np.all(board != 0).item()


print("Warming up")
# Warm-up Numba & python functions
connect4._numba_calculate_winner(initial_state.board, 0, 2, current_player, game.height, game.width, game.connect_length)
connect4._numba_calculate_is_board_full(initial_state.board)
py_calculate_winner(initial_state.board, 0, 2, current_player, game.height, game.width, game.connect_length)
py_calculate_is_board_full(initial_state.board)


## Numba version if ~6x faster.
# print("Profiling _calculate_winner:")
# %timeit connect4._numba_calculate_winner(initial_state.board, 0, 2, current_player, game.height, game.width, game.connect_length)
# %timeit py_calculate_winner(initial_state.board, 0, 2, current_player, game.height, game.width, game.connect_length)

# print("\nProfiling _calculate_is_board_full:")
# %timeit connect4._numba_calculate_is_board_full(initial_state.board)
# %timeit py_calculate_is_board_full(initial_state.board)
